In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state = 1)

In [3]:
import numpy as np
y_train = y_train.reshape(-1,1)
x_train=np.hstack((x_train,y_train))


In [5]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [6]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [8]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, examples):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = examples[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [9]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [10]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [11]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [12]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)
             
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question,column_split = gain, question,col

    return best_gain, best_question,col,current_uncertainty

In [13]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [14]:
def build_tree(rows,i):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question,col,entropy = find_best_split(rows)
    print("Level %d"%(i))
    print("entropy %f"%(entropy))
  
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        print(class_counts(rows))
        print("reached leaf node")
        return Leaf(rows)
    else :
          print(class_counts(rows))
          print("spliting on feature %s with gain %f:"%(col,gain))

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows,i+1)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows,i+1)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [15]:
my_tree = build_tree(x_train,0)


Level 0
entropy 0.664700
{1.0: 34, 2.0: 41, 0.0: 37}
spliting on feature 3 with gain 0.332795:
Level 1
entropy 0.495644
{1.0: 34, 2.0: 41}
spliting on feature 3 with gain 0.374544:
Level 2
entropy 0.051247
{2.0: 37, 1.0: 1}
spliting on feature 3 with gain 0.011773:
Level 3
entropy 0.000000
{2.0: 34}
reached leaf node
Level 3
entropy 0.375000
{2.0: 3, 1.0: 1}
spliting on feature 3 with gain 0.375000:
Level 4
entropy 0.000000
{1.0: 1}
reached leaf node
Level 4
entropy 0.000000
{2.0: 3}
reached leaf node
Level 2
entropy 0.192841
{1.0: 33, 2.0: 4}
spliting on feature 3 with gain 0.149598:
Level 3
entropy 0.320000
{2.0: 4, 1.0: 1}
spliting on feature 3 with gain 0.120000:
Level 4
entropy 0.500000
{1.0: 1, 2.0: 1}
spliting on feature 3 with gain 0.500000:
Level 5
entropy 0.000000
{2.0: 1}
reached leaf node
Level 5
entropy 0.000000
{1.0: 1}
reached leaf node
Level 4
entropy 0.000000
{2.0: 3}
reached leaf node
Level 3
entropy 0.000000
{1.0: 32}
reached leaf node
Level 1
entropy 0.000000
{0.0: 